<a href="https://colab.research.google.com/github/MarciaFG/skill-flow/blob/main/Flows_1980_2000_first_level_for.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Academic Mobility Flows using BigQuery**

Author: Marcia R. Ferreira (Complexity Science Hub Vienna & TU Wien)

Date: September 28, 2022

Input: Dimensions database on BigQuery

Output: GBQ Dimensions.ai

Other notes: 
*   To create the basic tables for all years copy this code above and rerun it for the next few decades;
*   Note that the tables need to have overlapping years otherwise it will not be possible to capture the transitions at year ceilings;


## Colab Initialization

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime → "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Mon Mar 13 11:05:41 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    51W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Install required Drivers

In [2]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
#!pip install psutil
#!pip install humanize
#!pip install pynput

# libraries
import psutil
import humanize
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import requests
import torch
import nltk
import GPUtil as GPU

# plotting
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

from google.cloud import bigquery
from google.colab import files
%load_ext google.colab.data_table
%load_ext google.cloud.bigquery

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for gputil: filename=GPUtil-1.4.0-py3-none-any.whl size=7409 sha256=93159348a56953c61b7247a783240426572c8a9e19e845923377b859e6207615
  Stored in directory: /root/.cache/pip/wheels/2b/b5/24/fbb56595c286984f7315ee31821d6121e1b9828436021a88b3
Successfully built gputil


In [3]:
# only one GPU on Colab and isn’t guaranteed
import psutil
import os
import humanize
import GPUtil as GPU

GPUs = GPU.getGPUs()
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ),\
       " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB"\
       .format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Gen RAM Free: 87.6 GB  | Proc size: 450.7 MB
GPU RAM Free: 40513MB | Used: 0MB | Util   0% | Total 40960MB


**Loading data from Google Drive (If needed)**

In [ ]:
# run this to upload files
# from google.colab import files
# uploaded = files.upload() 

**Mounting the Google Drive folder**

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

# let's test it
#with open('/content/drive/My Drive/foo.txt', 'w') as f:
#  f.write('Hello Google Drive!')
#!cat /content/drive/My\ Drive/foo.txt

Mounted at /content/drive
Hello Google Drive!

**Runtime credentials**

In [4]:
# Provide your credentials to the runtime
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


**Declare the Cloud project ID which will be used throughout this notebook**

In [5]:
# declare your project 
project_id = "cshdimensionstest"

# **PART I - Data Wrangling**

## 1.0 Load Data from GBQ

### *Basic Table*

1.   The intermediary table is restricted to:
  - publication id
  - researcher id
  - research orgs
  - first level for code
  - and year as our unit time
2.   Filters:
  - only disambiguated researchers for which author-affiliation linkages exist
  - research organizations for which a grid id exists
  - only publications that have the two things above + a first level code assigned to it
  - and researchers whose first publication has been after or in 1980
3.   Time period:
  - 1980-2000
  - we need to run different scripts for shorter time frames for easing the computational resources
  - we will consider paralelizing the full code at the end



In [ ]:
# Constructing the mobility flows intermediary table for the FOR categorization
%%bigquery --project $project_id 

#create or replace table cshdimensionstest.test.disambiguated_authors and corresponding publications
CREATE OR REPLACE TABLE cshdimensionstest.test.basic_1980_2000 AS 

SELECT p.id, researcher_ids, research_orgs, category_for.code, p.year
FROM `dimensions-ai.data_analytics.publications` p
    , unnest(category_for.first_level.full) category_for
    , unnest(researcher_ids) researcher_ids
    , unnest(research_orgs) research_orgs
    JOIN `dimensions-ai.data_analytics.researchers` r 
    ON r.id=p.researcher_ids
WHERE researcher_ids IS NOT NULL 
  AND research_orgs IS NOT NULL
  AND category_for IS NOT NULL -- its best to allow for null values here
  AND p.year BETWEEN 1980 AND 2000
  AND first_publication >= 1980
ORDER BY p.id, researcher_ids, research_orgs

-- this gives us the publications with disambiguated researchers ids
-- AND  the pubs with authors that have affiliation linkages
-- AND the pubs with author-aff links that have an FOR category associated
-- AND between 1980 and 2000
-- This will be our basic table

In [6]:
%%bigquery --project $project_id
-- let's have a look
SELECT  * FROM cshdimensionstest.test.basic_1980_2000 limit 2;

Query is running:   0%|          |

Downloading:   0%|          |

,id,researcher_ids,research_orgs,year,is_multi_affiliation,aff_w
0,pub.1016133979,ur.013447613731.39,grid.462226.6,1980,1,10
1,pub.1036727738,ur.015357527543.00,grid.6906.9,1980,1,14


In [7]:
%%bigquery --project $project_id
-- count the number of FOR categories per publication 
-- SELECT id, COUNT (DISTINCT code) N_codes FROM cshdimensionstest.test.basic_1980_2000 GROUP BY id order by N_CODES DESC LIMIT 5;
-- A publication can have up to 5 codes
SELECT COUNT(*) FROM cshdimensionstest.test.basic_1980_2000;
-- 55873771 total rows

Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,55873771


In [ ]:
%%bigquery --project $project_id
SELECT COUNT(distinct researcher_ids) FROM cshdimensionstest.test.basic_1980_2000; -- 4,510,186 distinct researchers
SELECT COUNT(distinct research_orgs) FROM cshdimensionstest.test.basic_1980_2000; -- 35,136 unique organizations
SELECT COUNT(distinct id) FROM cshdimensionstest.test.basic_1980_2000; -- 10,025,338 publications of any document type

### *Multiple Affiliations* - NOT RUN

* Identify multiple affiliations in the table and update the basic table



In [ ]:
%%bigquery --project $project_id 
# indicate whether an author-affiliation is shared  or not
CREATE OR REPLACE TABLE cshdimensionstest.test.multi_affiliations AS

SELECT DISTINCT p.id, p.researcher_ids, p.research_orgs, p.year, s.is_multi_affiliation, s.aff_w
FROM cshdimensionstest.test.basic_1980_2000 p
 JOIN 
    (
    SELECT id, researcher_ids, COUNT(DISTINCT research_orgs) as aff_w, CASE WHEN COUNT(DISTINCT research_orgs)  > 1 Then 1 Else 0 END is_multi_affiliation
    FROM cshdimensionstest.test.basic_1980_2000
    GROUP BY id, researcher_ids
    ) s
  ON p.id=s.id and p.researcher_ids=s.researcher_ids;

CREATE OR REPLACE TABLE cshdimensionstest.test.basic_1980_2000 AS SELECT * FROM cshdimensionstest.test.multi_affiliations;

SELECT * FROM cshdimensionstest.test.basic_1980_2000 order by id, researcher_ids, research_orgs LIMIT 20;

DROP TABLE IF EXISTS cshdimensionstest.test.multi_affiliations;

## 2.0 Researcher Trajectories

#### *Time Sequences*

In [ ]:
%%bigquery --project $project_id 

# step (1): give a row number to the years in the order
 create or replace table cshdimensionstest.test.sequence_1980_2000 as 
  select distinct researcher_ids, 
    year, 
    dense_rank() over (
      partition by researcher_ids 
      order by 
        year asc
    ) as t 
  from `cshdimensionstest.test.basic_1980_2000`
  order by 
    researcher_ids, 
    year, 
    t;

In [8]:
%%bigquery --project $project_id 
SELECT * FROM cshdimensionstest.test.sequence_1980_2000 where researcher_ids = 'ur.011460612366.60' order by t;

Query is running:   0%|          |

Downloading:   0%|          |

,researcher_ids,year,t
0,ur.011460612366.60,1980,1
1,ur.011460612366.60,1981,2
2,ur.011460612366.60,1982,3
3,ur.011460612366.60,1983,4
4,ur.011460612366.60,1985,5
5,ur.011460612366.60,1986,6
6,ur.011460612366.60,1987,7
7,ur.011460612366.60,1988,8
8,ur.011460612366.60,1989,9
9,ur.011460612366.60,1990,10


#### *Affiliation Weights*

In [ ]:
%%bigquery --project $project_id 
# step (2)
# generating affiliation weights if the author has had more than one affiliation simultaneously
 create or replace table cshdimensionstest.test.affweight_1980_2000 as 
  select 
    distinct researcher_ids, 
    id, 
    1 * 1.0 / count(distinct research_orgs) as aff_weight -- this weight is at the publication level
  from 
    `cshdimensionstest.test.basic_1980_2000`
  group by 
    researcher_ids, 
    id
  order by researcher_ids, id;

drop table if exists cshdimensionstest.test.affweight_00_02;

In [9]:
%%bigquery --project $project_id 
SELECT * FROM cshdimensionstest.test.affweight_1980_2000 order by researcher_ids, id limit 10;

Query is running:   0%|          |

Downloading:   0%|          |

,researcher_ids,id,aff_weight
0,ur.010000001271.33,pub.1039310092,1.000000
1,ur.010000001341.07,pub.1064721680,0.333333
2,ur.01000000143.58,pub.1007920533,0.500000
3,ur.01000000143.58,pub.1055163401,1.000000
4,ur.01000000162.06,pub.1017303151,1.000000
5,ur.01000000162.06,pub.1040922518,1.000000
6,ur.01000000162.06,pub.1082688278,1.000000
7,ur.010000001625.53,pub.1000729400,1.000000
8,ur.010000001625.53,pub.1020269617,1.000000
9,ur.01000000255.40,pub.1010721240,0.500000


In [ ]:
%%bigquery --project $project_id 
# step (3)
# merging results from steps 1-2
create or replace table cshdimensionstest.test.psequence_weight_1980_2000 as 
  select 
      a.researcher_ids,
      a.id,
    --  a.code,
      a.year,
      a.research_orgs,
      a.is_multi_affiliation,
      b.t,
      c.aff_weight, 
      a.aff_w as n_au_orgs
  from
      `cshdimensionstest.test.basic_1980_2000` as a 
      inner join
         `cshdimensionstest.test.sequence_1980_2000` as b 
         on a.researcher_ids = b.researcher_ids 
         and a.year = b.year 
      inner join
         `cshdimensionstest.test.affweight_1980_2000` as c 
         on c.researcher_ids = a.researcher_ids 
         and c.id = a.id 
  order by
        b.researcher_ids,
        b.year,
        b.t;

  # drop table if exists cshdimensionstest.test.psequence_weight_00_02;

  select * from cshdimensionstest.test.psequence_weight_1980_2000  order by researcher_ids, id, year, t limit 10;

Query is running:   0%|          |

Downloading:   0%|          |

,researcher_ids,id,year,research_orgs,is_multi_affiliation,t,aff_weight,n_au_orgs
0,ur.010000001271.33,pub.1039310092,1985,grid.417643.3,0,1,1.000000,1
1,ur.010000001341.07,pub.1064721680,1998,grid.224260.0,1,1,0.333333,3
2,ur.010000001341.07,pub.1064721680,1998,grid.265457.7,1,1,0.333333,3
3,ur.010000001341.07,pub.1064721680,1998,grid.448385.6,1,1,0.333333,3
4,ur.01000000143.58,pub.1007920533,2000,grid.17091.3e,1,1,0.500000,2
5,ur.01000000143.58,pub.1007920533,2000,grid.417570.0,1,1,0.500000,2
6,ur.01000000143.58,pub.1055163401,2000,grid.17091.3e,0,1,1.000000,1
7,ur.01000000162.06,pub.1017303151,1993,grid.412587.d,0,1,1.000000,1
8,ur.01000000162.06,pub.1040922518,1993,grid.412587.d,0,1,1.000000,1
9,ur.01000000162.06,pub.1082688278,1993,grid.412597.c,0,1,1.000000,1


#### *First Affiliation*


*   The first affiliation of an author is sometimes in the data table. This has to do with (1) missing author-affiliation linkages and/or (2) lack of field classification codes associated to a publication
*   The column `is_orign` marks whether that institution is the authors' first affiliation in the whole database and not just in the dataset for the overall period 1980-2022



In [ ]:
%%bigquery --project $project_id 
# filter the dataset by researchers that started in 1980 or after
# all researchers can still be found in this table cshdimensionstest.test.psequence_weight_00_02
create or replace table cshdimensionstest.test.researchers_after_1980 as
  select p.*
    , first_publication_year
    , case 
        when first_publication_year = year then 1 else 0 
      end is_origin
  from cshdimensionstest.test.psequence_weight_1980_2000 p
  join (
        select distinct researcher_ids, first_publication_year
        from cshdimensionstest.test.psequence_weight_1980_2000 au
        left join dimensions-ai.data_analytics.researchers r on au.researcher_ids=r.id
        where first_publication_year >= 1980
        ) s
    on p.researcher_ids=s.researcher_ids;
    
select * from cshdimensionstest.test.researchers_after_1980 
order by researcher_ids,  year, t limit 10;

Query is running:   0%|          |

Downloading:   0%|          |

,researcher_ids,id,year,research_orgs,is_multi_affiliation,t,aff_weight,n_au_orgs,first_publication_year,is_origin
0,ur.010000001271.33,pub.1039310092,1985,grid.417643.3,0,1,1.000000,1,1981,0
1,ur.010000001341.07,pub.1064721680,1998,grid.224260.0,1,1,0.333333,3,1997,0
2,ur.010000001341.07,pub.1064721680,1998,grid.265457.7,1,1,0.333333,3,1997,0
3,ur.010000001341.07,pub.1064721680,1998,grid.448385.6,1,1,0.333333,3,1997,0
4,ur.01000000143.58,pub.1055163401,2000,grid.17091.3e,0,1,1.000000,1,2000,1
5,ur.01000000143.58,pub.1007920533,2000,grid.417570.0,1,1,0.500000,2,2000,1
6,ur.01000000143.58,pub.1007920533,2000,grid.17091.3e,1,1,0.500000,2,2000,1
7,ur.01000000162.06,pub.1040922518,1993,grid.412587.d,0,1,1.000000,1,1993,1
8,ur.01000000162.06,pub.1082688278,1993,grid.412597.c,0,1,1.000000,1,1993,1
9,ur.01000000162.06,pub.1017303151,1993,grid.412587.d,0,1,1.000000,1,1993,1


In [ ]:
%%bigquery --project $project_id 
# make a list of all origins and researcher_ids combinations in the dataset
# match the origins to the whole trajectory and mark it as 1
create or replace table cshdimensionstest.test.origins as
select distinct researcher_ids, research_orgs, is_origin
from cshdimensionstest.test.researchers_after_1980
where is_origin = 1;

In [ ]:
%%bigquery --project $project_id 
# join all the origins to the trajectories after 1980 table
create or replace table cshdimensionstest.test.researchers_after_1980_with_origins as
select a.*, b.research_orgs as first_affiliation, ifnull(b.is_origin, 0) is_origin_all
from cshdimensionstest.test.researchers_after_1980 a
left join cshdimensionstest.test.origins b 
  on a.researcher_ids=b.researcher_ids
  and a.research_orgs=b.research_orgs;

drop table cshdimensionstest.test.researchers_after_1980;
create or replace table cshdimensionstest.test.researchers_after_1980 as
select * from cshdimensionstest.test.researchers_after_1980_with_origins;
drop table cshdimensionstest.test.researchers_after_1980_with_origins;
ALTER TABLE cshdimensionstest.test.researchers_after_1980 DROP COLUMN is_multi_affiliation;
ALTER TABLE cshdimensionstest.test.researchers_after_1980 DROP COLUMN first_affiliation;
ALTER TABLE cshdimensionstest.test.researchers_after_1980 DROP COLUMN first_publication_year;
ALTER TABLE cshdimensionstest.test.researchers_after_1980 DROP COLUMN is_origin;

In [ ]:
%%bigquery --project $project_id 

create or replace table cshdimensionstest.test.researchers_after_1980_simplified as
with data_simplified as 
  (
  select distinct researcher_ids, research_orgs, year, t, is_origin_all
  from cshdimensionstest.test.researchers_after_1980
  ), affiliation_weights as (
    SELECT researcher_ids, year, t, is_origin_all, 1 / COUNT(DISTINCT research_orgs) AS aff_weight --> we use this weight in constructing the network
    FROM data_simplified
    GROUP BY researcher_ids, year, t, is_origin_all
    ) 
SELECT distinct b.researcher_ids, b.research_orgs, b.year, b.t, b.is_origin_all, a.aff_weight 
FROM affiliation_weights a
INNER JOIN cshdimensionstest.test.researchers_after_1980 b 
ON a.researcher_ids=b.researcher_ids AND a.year=b.year AND a.t=b.t;

select * 
from cshdimensionstest.test.researchers_after_1980_simplified 
order by researcher_ids, t
limit 10

Query is running:   0%|          |

Downloading:   0%|          |

,researcher_ids,research_orgs,year,t,is_origin_all,aff_weight
0,ur.010000001271.33,grid.417643.3,1985,1,0,1.000000
1,ur.010000001341.07,grid.448385.6,1998,1,0,0.333333
2,ur.010000001341.07,grid.265457.7,1998,1,0,0.333333
3,ur.010000001341.07,grid.224260.0,1998,1,0,0.333333
4,ur.01000000143.58,grid.417570.0,2000,1,1,0.500000
5,ur.01000000143.58,grid.17091.3e,2000,1,1,0.500000
6,ur.01000000162.06,grid.412587.d,1993,1,1,0.500000
7,ur.01000000162.06,grid.412597.c,1993,1,1,0.500000
8,ur.010000001625.53,grid.5596.f,1998,1,1,0.500000
9,ur.010000001625.53,grid.498578.f,1998,1,1,0.500000


**Ok now we can contruct the mobility network.**


---

- we can also use this table to calculate the number of publications of the author per `year`
- we use the publications in this table to calculate the indicators for authors for the period 1980-2000
- we count the fractional number of papers using the `aff_weight`
- note that the table contains repeated rows for author-pub-org combinations

# **PART II - Mobility Networks & Indicators**

## 3.0 Network Flows

We will split the calculation of the network flows:
1. Institutions
2. Cities
3. Countries

Mobility means co-occuring affiliations. Does not mean actual mobility.
Establishing affiliation connections.

---



We can later on think of costumisable layers such as NUTS2 etc

### 3.1 Cross-Institutional Flows
In this part we focus on flows at the level of institutions

In [ ]:
%%bigquery --project $project_id 
# now we have everything we need to construct the flows at the institutional level
create or replace table cshdimensionstest.test.flows_1980_2000 as 
  select distinct
    a.researcher_ids,
    a.research_orgs as unit1,
    b.research_orgs as unit2,
    a.t as t1,
    b.t as t2,
    a.year as p1,
    b.year as p2,
    a.aff_weight as w1,
    b.aff_weight as w2,
    a.is_origin_all as origin1,
    b.is_origin_all as origin2
  from
        cshdimensionstest.test.researchers_after_1980_simplified a 
    inner join
        cshdimensionstest.test.researchers_after_1980_simplified b 
        on a.researcher_ids = b.researcher_ids 
  where
        a.t < b.t and a.t = b.t - 1;

# check the table
select * 
from cshdimensionstest.test.flows_1980_2000 
order by researcher_ids, t1, t2, unit1, unit2 
limit 50;

Query is running:   0%|          |

Downloading:   0%|          |

,researcher_ids,unit1,unit2,t1,t2,p1,p2,w1,w2,origin1,origin2
0,ur.01000000255.40,grid.136593.b,grid.136593.b,1,2,1980,1981,0.5,1.000000,1,1
1,ur.01000000255.40,grid.136593.b,grid.258799.8,1,2,1980,1981,0.5,1.000000,1,0
2,ur.01000000255.40,grid.416963.f,grid.136593.b,1,2,1980,1981,0.5,1.000000,1,1
3,ur.01000000255.40,grid.416963.f,grid.258799.8,1,2,1980,1981,0.5,1.000000,1,0
4,ur.01000000352.51,grid.10253.35,grid.10253.35,1,2,1993,1994,1.0,1.000000,1,1
5,ur.01000000352.51,grid.10253.35,grid.6553.5,1,2,1993,1994,1.0,1.000000,1,0
6,ur.01000000352.51,grid.10253.35,grid.10253.35,2,3,1994,1997,1.0,1.000000,1,1
7,ur.01000000352.51,grid.6553.5,grid.10253.35,2,3,1994,1997,1.0,1.000000,0,1
8,ur.01000000367.29,grid.47840.3f,grid.47840.3f,1,2,1989,1991,1.0,1.000000,1,1
9,ur.01000000367.29,grid.47840.3f,grid.47840.3f,2,3,1991,1994,1.0,1.000000,1,1


*   ATT: This is the most computationally expensive table, becareful with running it too many times
* now we are ready to aggregate the flows:

### **3.2 Indicators**

# CONTINUE HERE

###**3.2.1 Source-Target Flows**


 #### *Total Flows (Pairs)*
 
 Are the flows exchanged between two instituions at a given calendar year

In [ ]:
%%bigquery --project $project_id

-- Calculate the total flows between institutional pairs
CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2000_institutional_flows AS
SELECT 
  unit1 AS geoid_o,
  unit2 AS geoid_d,
  p2 AS date_d,
  COUNT(DISTINCT researcher_ids) AS total_flows, # author flows
 -- SUM(w1) as w1,
 -- SUM(w2) as w2,
 -- origin1,
 -- origin2
FROM 
  cshdimensionstest.test.flows_1980_2000
GROUP BY 
  geoid_o, 
  geoid_d, 
  date_d;

-- Check the table 
SELECT * 
FROM cshdimensionstest.test.flows_1980_2000_institutional_flows 
ORDER BY 
  geoid_o, 
  geoid_d, 
  date_d 
LIMIT 50;


In [23]:
%%bigquery --project $project_id

# remove self-loops from total flows table
CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2000_institutional_flows_agg AS
select * 
from cshdimensionstest.test.flows_1980_2000_institutional_flows
where geoid_d != geoid_o;

# check  table 
SELECT * 
FROM cshdimensionstest.test.flows_1980_2000_institutional_flows_agg
ORDER BY 
  geoid_o, 
  geoid_d, 
  date_d 
LIMIT 50;


Query is running:   0%|          |

Downloading:   0%|          |

,geoid_o,geoid_d,date_d,total_flows
0,grid.1001.0,grid.1002.3,1981,3
1,grid.1001.0,grid.1002.3,1982,9
2,grid.1001.0,grid.1002.3,1983,6
3,grid.1001.0,grid.1002.3,1984,16
4,grid.1001.0,grid.1002.3,1985,8
5,grid.1001.0,grid.1002.3,1986,20
6,grid.1001.0,grid.1002.3,1987,18
7,grid.1001.0,grid.1002.3,1988,25
8,grid.1001.0,grid.1002.3,1989,23
9,grid.1001.0,grid.1002.3,1990,47


In [65]:
%%bigquery --project $project_id

CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2000_institutional_total_flows_agg AS
SELECT
  node,
  date_d,
  SUM(outgoing_flows) AS outgoing_flows,
  ROUND(SUM(outgoing_flows) / (SUM(outgoing_flows) + SUM(incoming_flows)) * 100, 1) AS percentage_outflows,
  SUM(incoming_flows) AS incoming_flows,
  ROUND(SUM(incoming_flows) / (SUM(outgoing_flows) + SUM(incoming_flows)) * 100, 1) AS percentage_inflows,
  SUM(outgoing_flows) + SUM(incoming_flows) AS total_flows,
  SUM(incoming_flows) - SUM(outgoing_flows) AS total_net_flows,
  ROUND((SUM(incoming_flows) - SUM(outgoing_flows)) / (SUM(incoming_flows) + SUM(outgoing_flows)) * 100, 1) AS net_migration_rate
FROM (
  SELECT
    unit1 AS node,
    p2 as date_d,
    COUNT(DISTINCT researcher_ids) AS outgoing_flows,
    0 AS incoming_flows
  FROM
    cshdimensionstest.test.flows_1980_2000
    WHERE unit1 != unit2
  GROUP BY
    unit1, p2

  UNION ALL

  SELECT
    unit2 AS node,
    p2 as date_d,
    0 AS outgoing_flows,
    COUNT(DISTINCT researcher_ids) AS incoming_flows
  FROM
    cshdimensionstest.test.flows_1980_2000
  GROUP BY
    unit2, p2

) AS flows
GROUP BY
  node, date_d;


  -- Check the table 
SELECT * 
FROM cshdimensionstest.test.flows_1980_2000_institutional_total_flows_agg
ORDER BY 
  node, 
  date_d 
LIMIT 50;

Query is running:   0%|          |

Downloading:   0%|          |

,node,date_d,outgoing_flows,percentage_outflows,incoming_flows,percentage_inflows,total_flows,total_net_flows,net_migration_rate
0,grid.1001.0,1981,43,37.4,72,62.6,115,29,25.2
1,grid.1001.0,1982,50,25.9,143,74.1,193,93,48.2
2,grid.1001.0,1983,94,32.5,195,67.5,289,101,34.9
3,grid.1001.0,1984,156,35.9,278,64.1,434,122,28.1
4,grid.1001.0,1985,195,37.4,326,62.6,521,131,25.1
5,grid.1001.0,1986,210,36.5,365,63.5,575,155,27.0
6,grid.1001.0,1987,248,34.8,465,65.2,713,217,30.4
7,grid.1001.0,1988,370,39.5,567,60.5,937,197,21.0
8,grid.1001.0,1989,381,38.1,618,61.9,999,237,23.7
9,grid.1001.0,1990,483,41.2,688,58.8,1171,205,17.5


#### *Top K destinations for each origin node*

In [ ]:
%%bigquery --project $project_id

# top 5 mobility destinations for each node
CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2000_institutional_top_k_destinations AS
SELECT
  origin_node,
  date_d,
  destination_node,
  incoming_flows,
  top_rank
FROM (
  SELECT
    origin_node,
    date_d,
    destination_node,
    incoming_flows,
    ROW_NUMBER() OVER (PARTITION BY origin_node, date_d ORDER BY incoming_flows DESC) AS top_rank
  FROM (
    SELECT
      unit1 AS origin_node,
      unit2 AS destination_node,
      p2 AS date_d,
      COUNT(DISTINCT researcher_ids) AS incoming_flows
    FROM
      cshdimensionstest.test.flows_1980_2000
    WHERE
      unit1 != unit2 
    GROUP BY
      origin_node, destination_node, date_d
  ) AS flows
) AS ranked_flows
WHERE
  top_rank <= 5
ORDER BY
  origin_node, date_d, top_rank;

In [ ]:
%%bigquery --project $project_id
# top 5 mobility sources for each node
CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2000_institutional_top_k_sources AS
SELECT
  origin_node,
  date_d,
  destination_node,
  outgoing_flows ,
  top_rank
FROM (
  SELECT
    destination_node,
    date_d,
    origin_node,
    outgoing_flows,
    ROW_NUMBER() OVER (PARTITION BY destination_node, date_d ORDER BY outgoing_flows DESC) AS top_rank
  FROM (
    SELECT
      unit2 AS destination_node,
      unit1 AS origin_node,
      p2 AS date_d,
      COUNT(DISTINCT researcher_ids) AS outgoing_flows
    FROM
      cshdimensionstest.test.flows_1980_2000
    WHERE
      unit1 != unit2
    GROUP BY
      origin_node, destination_node, date_d
  ) AS flows
) AS ranked_flows
WHERE
  top_rank <= 5
ORDER BY
  destination_node, date_d, top_rank;


In [ ]:
%%bigquery --project $project_id

SELECT * FROM cshdimensionstest.test.flows_1980_2000_institutional_top_k_sources
ORDER BY  destination_node, date_d, top_rank
LIMIT 50;

SELECT * FROM cshdimensionstest.test.flows_1980_2000_institutional_top_k_destinations
ORDER BY  destination_node, date_d, top_rank
LIMIT 50;

#### *Average duration of mobility flows*
Description: The average year of mobility flows between each node pair in years

The `avg_year` calculated in the query provides the average year of migration flow between each node pair. This can be useful in several ways:

**Trend analysis**: The average year can be used to track the trend of migration flows between different pairs of nodes over time. For example, if the average year for a particular node pair shows a steady increase over time, it may suggest that migration flows between those nodes are becoming more frequent or more stable.

**Migration patterns**: The average year can also provide insights into the patterns of migration flows between different pairs of nodes. For example, if the average year for a node pair is relatively low, it may indicate that migration flows between those nodes are more likely to be short-term or seasonal, while a higher average year may suggest longer-term migration flows.

**Regional comparisons**: Comparing the average year between different node pairs can also reveal differences in migration patterns and trends between different regions or areas. This information can be useful for policymakers, researchers, and other stakeholders who are interested in understanding migration patterns and trends in different areas and their potential drivers.

Overall, the average year can be a useful metric for understanding the temporal dynamics of migration flows between different node pairs and can provide valuable insights into migration patterns, trends, and regional differences.

In [110]:
 %%bigquery --project $project_id
CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2000_avg_year AS
 SELECT
  unit1 as geoid_o,
  unit2 as geoid_d,
  ROUND(AVG((p1 + p2) / 2.0), 1) AS avg_year
FROM
 cshdimensionstest.test.flows_1980_2000 
 WHERE unit1!=unit2
GROUP BY
  unit1, unit2;

SELECT * FROM cshdimensionstest.test.flows_1980_2000_avg_year
ORDER BY  geoid_o, geoid_d, avg_year
LIMIT 50;

Query is running:   0%|          |

Downloading:   0%|          |

,geoid_o,geoid_d,avg_year
0,grid.1001.0,grid.1002.3,1992.5
1,grid.1001.0,grid.10025.36,1995.7
2,grid.1001.0,grid.1003.2,1993.5
3,grid.1001.0,grid.1004.5,1992.3
4,grid.1001.0,grid.10041.34,1999.1
5,grid.1001.0,grid.10049.3c,1995.5
6,grid.1001.0,grid.1005.4,1993.5
7,grid.1001.0,grid.1006.7,1992.5
8,grid.1001.0,grid.1007.6,1992.7
9,grid.1001.0,grid.1008.9,1993.6


In [ ]:
%%bigquery --project $project_id

SELECT * FROM cshdimensionstest.test.flows_1980_2000_institutional_top_k_destinations
ORDER BY  origin_node, date_d, top_rank
LIMIT 50;

#### *Total Outflows (Pairs)*
In the context of mobility, "outflows from pairs" could refer to the movement of people away from a pair or group traveling together.

 #### *Net Flows (Pairs)*


In [36]:
%%bigquery --project $project_id

-- Calculate the total net flows between institutional pairs
CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2000_institutional_net_flows AS
SELECT 
  a.geoid_o, # source
  a.geoid_d, # target
  a.date_d,     # move date
  a.total_flows as total_flows_pair, # total people moving between the  pairs
  total_inflows as total_inflows_pair,    # inflow counts
 -- total_inflows / total_flows * 100 as percentage_inflows_pair,
  total_outflows as total_outflows_pair,   # outflow counts
--  total_outflows / total_flows * 100 as percentage_outflows_pair,
  total_inflows - total_outflows as net_flows_pair, # net flows
FROM 
  cshdimensionstest.test.flows_1980_2000_institutional_flows_agg a
  LEFT JOIN cshdimensionstest.test.flows_1980_2000_institutional_inflows i 
    on i.geoid_d=a.geoid_d
    AND i.date_d=a.date_d
  LEFT JOIN cshdimensionstest.test.flows_1980_2000_institutional_outflows o
    on o.geoid_o=a.geoid_o
    AND o.date_d=a.date_d
GROUP BY  a.geoid_o
    , a.geoid_d
    , a.date_d
    , total_inflows
    , total_outflows;

  -- Check the table 
SELECT * FROM cshdimensionstest.test.flows_1980_2000_institutional_net_flows
ORDER BY 
  geoid_o,
  geoid_d, 
  date_d 
LIMIT 50;

Executing query with job ID: a795adf6-533d-4787-be9e-1d1f87362a60
Query executing: 2.09s


ERROR:
 400 Query error: Name total_flows_in not found inside i at [8:9]

Location: US
Job ID: a795adf6-533d-4787-be9e-1d1f87362a60



 #### *Net Migration Rate (Pairs)*

### **3.2.2 Population Flows**

#### *Total Outflows*
**Outflows**: Total number of researchers leaving an institution at a given calendar year
Note that the outflow is given by the last year of publication

In [ ]:
%%bigquery --project $project_id

# calculate the total outflows by calendar year
CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2000_institutional_outflows AS
SELECT unit1 AS geoid_o,
   --    unit2 AS geoid_d,
       p1 AS date_o, --> to count the outflows we need the last publishing date
  --   p2 AS date_d,
       COUNT(DISTINCT researcher_ids) outflows
FROM cshdimensionstest.test.flows_1980_2000
GROUP BY geoid_o, date_o;

# check table 
select * 
from cshdimensionstest.test.flows_1980_2000_institutional_outflows 
order by geoid_o, date_o
limit 5;  # limit the result to 5 rows for viewing purposes

Query is running:   0%|          |

Downloading:   0%|          |

,geoid_o,date_d,outflows
0,grid.1001.0,1981,25
1,grid.1001.0,1982,40
2,grid.1001.0,1983,76
3,grid.1001.0,1984,138
4,grid.1001.0,1985,160


#### *Total Inflows*
 **Inflows**: Total number of researchers entering an institution in a given calendar year

In [ ]:
# Define the BigQuery project
%%bigquery --project $project_id

# Calculate the total inflows
CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2000_institutional_inflows AS
SELECT 
  -- unit1 AS geoid_o,   # commented out as it is not being used in the query
      unit2 AS geoid_d,  # rename column unit2 to geoid_d
  -- p1 AS date_o,       # commented out as it is not being used in the query
       p2 AS date_d,    # rename column p2 to date_d
       COUNT(DISTINCT researcher_ids) inflows  # count the number of unique researcher_ids
FROM cshdimensionstest.test.flows_1980_2000
GROUP BY geoid_d, date_d;  # group by destination geoid and date

# Check the created table
SELECT * 
FROM cshdimensionstest.test.flows_1980_2000_institutional_inflows 
ORDER BY geoid_d, date_d 
LIMIT 5;  # limit the result to 5 rows for viewing purposes

Query is running:   0%|          |

Downloading:   0%|          |

,geoid_d,date_d,inflows
0,grid.1001.0,1981,12
1,grid.1001.0,1982,46
2,grid.1001.0,1983,75
3,grid.1001.0,1984,106
4,grid.1001.0,1985,146


 #### *Net Migration Rate*

**Net migration rate**: the difference between the number of people moving into a place and the number of people moving out of that place, expressed as a proportion of the population in a given calendar year

<p>Mathematically: N = (I - O) / P * 100</p>
<p>Where:</p>
<ul>
  <li>N = Net migration rate</li>
  <li>I = Number of people moving into a place (inflows)</li>
  <li>O = Number of people moving out of a place (outflows)</li>
  <li>P = Population</li>
</ul>


In [ ]:
# to calculate the net migration rate we need to know the population first
# we count all publishing authors in the dataset
%%bigquery --project $project_id
create or replace table cshdimensionstest.test.population_1980_2000 as
select research_orgs, year, count(distinct researcher_ids) as population_year
from cshdimensionstest.test.researchers_after_1980_simplified 
group by research_orgs, year;

In [ ]:
%%bigquery --project $project_id
# Calculate the the inflows and outflows table
CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2000_institution_flows_indicators AS
SELECT ifnull(geoid_o, geoid_d) as org
, ifnull(a.date_d, b.date_d) as date_d --> CHANGE THIS
, ifnull(inflows, 0) as inflows
, ifnull(outflows, 0) as outflows
FROM cshdimensionstest.test.flows_1980_2000_institutional_inflows a 
LEFT JOIN cshdimensionstest.test.flows_1980_2000_institutional_outflows b
ON a.geoid_d=b.geoid_o and a.date_d=b.date_d;

In [ ]:
%%bigquery --project $project_id
-- merge the population counts per year with the inflows and outflows table
-- calculate the net migration rate
CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2000_institutional_in_out_net AS
SELECT a.*, 
       b.inflows, 
       b.outflows, 
       ROUND((inflows - outflows) / population_year * 100, 5) AS net_migration_rate
FROM cshdimensionstest.test.population_1980_2000 a
LEFT JOIN cshdimensionstest.test.flows_1980_2000_institution_flows_indicators b 
ON b.org = a.research_orgs AND b.date_d = a.year;

-- Delete any redundant tables from GBQ
DROP TABLE IF EXISTS cshdimensionstest.test.flows_1980_2000_institutional_inflows;
DROP TABLE IF EXISTS cshdimensionstest.test.flows_1980_2000_institutional_outflows;
DROP TABLE IF EXISTS cshdimensionstest.test.flows_1980_2000_institution_flows_indicators;

In [ ]:
%%bigquery --project $project_id
SELECT * 
FROM cshdimensionstest.test.flows_1980_2000_institutional_in_out_net 
ORDER BY research_orgs, year 
LIMIT 5; 

Query is running:   0%|          |

Downloading:   0%|          |

,research_orgs,year,population_year,inflows,outflows,net_migration_rate
0,grid.1001.0,1980,202,<NA>,<NA>,NaN
1,grid.1001.0,1981,252,12,25,-5.15873
2,grid.1001.0,1982,339,46,40,1.76991
3,grid.1001.0,1983,392,75,76,-0.25510
4,grid.1001.0,1984,512,106,138,-6.25000


The net migration rate is an indicator that provides insight into the balance between the number of people moving into and out of a place. It is expressed as a percentage of the population, making it a useful measure for comparing the migration trends of different areas. A positive net migration rate indicates that there are more people moving into the place than moving out, while a negative rate indicates that there are more people leaving than arriving.

Interpretation of the net migration rate:

  *  A positive net migration rate indicates growth in the researcher population due to mobility, which can be seen as a positive sign for the research system, academic labour market, and overall development of the institution.

    *   A **high** positive net migration rate may indicate a strong pull factor, such as a growing economy, high quality of life, or attractive resources.

    *   A negative net migration rate indicates a decrease in the researcher population due to mobility, which may indicate that people are leaving the institution due to factors such as a declining funding, or a lack of career progression opportunities.

   *    A low negative net migration rate may indicate a minor trend of people leaving the institution, while a high negative rate may indicate a more serious and persistent trend of population loss at the institutional level.

It is important to note that the net migration rate is only one of many indicators that can be used to understand academic mobility trends, and it is important to consider other factors such as academic age, background, and overall status of the population in order to gain a full understanding of the mobility patterns in a given research institution.

# REVISION

#### *Absence of mobility*
To calculate the portion of the population that did not "move" by calendar year, I use the following formula:

Portion of the population that did not migrate is calculated as 
* `(total population - (total_inflows + total_outflows)) / total population * 100`

This will give the percentage of the population that remained in the institution and did not migrate in a given year.

In [ ]:
%%bigquery --project $project_id
-- Calculation the non-migrant percentage by year
CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2000_institutional_total_flow_indicators AS
WITH population_and_migrants AS
(
  SELECT 
    year, 
    research_orgs, 
    population_year, 
    inflows, 
    outflows, 
    net_migration_rate, 
    inflows + outflows AS total_migrants
  FROM cshdimensionstest.test.flows_1980_2000_institutional_in_out_net
)
SELECT 
  year, 
  research_orgs, 
  population_year, 
  inflows, 
  outflows, 
  net_migration_rate, 
  total_migrants, 
  ((population_year - total_migrants) / population_year) * 100 AS non_migrant_percentage
FROM population_and_migrants
ORDER BY research_orgs, year;

-- Delete any redundant tables from GBQ
DROP TABLE IF EXISTS cshdimensionstest.test.flows_1980_2000_institutional_in_out_net;

In [ ]:
%%bigquery --project $project_id
-- view the top 10 rows of the created table
SELECT * 
FROM cshdimensionstest.test.flows_1980_2000_institutional_total_flow_indicators
ORDER BY research_orgs, year
LIMIT 10;

#### *Retention Rates*

To calculate the Scientists' retention rate per calendar year, I follow these steps:

    1. Identify the number of scientists affiliated in each institution in each year.
    2. Identify the number of scientists who were affiliated in the same institution in the previous year.
    3. Calculate the retention rate as the number of scientists who remained in the same institution divided by the total number of scientists enrolled in that institution.
    4. Repeat the process for each year and each institution.

This procedure calculates the retention rate of scientists affiliated with an institution from one year to the next. The retention rate is defined as *the ratio of the number of scientists affiliated with an institution in a given year to the number of scientists affiliated with the same institution in the previous year*.

The interpretation of the retention rate:

    A value of 1 means that the institution was able to retain the same number of scientists as the previous year
    , while a value greater than 1 indicates an increase in the number of affiliated scientists
    , and a value less than 1 indicates a decrease. 
    The output can be used to assess the ability of institutions
    to retain their affiliated scientists over time

This code calculates the retention rate of scientists for each institution and year, which is defined as the proportion of scientists who are affiliated with the institution in the current year and were also affiliated with the same institution in the previous year.

Advantages of this indicator include:

    It provides a clear picture of how well institutions are retaining their existing scientists.
    It is a simple indicator that can be easily understood.

Limitations of this indicator include:

    It does not take into account new scientists who may have joined the institution in the current year, so it may not provide a complete picture of the institution's ability to attract scientists.
    It assumes that the number of scientists affiliated with an institution in any given year is a good proxy for the institution's ability to attract and retain scientists. This may not always be the case, as factors such as funding, working conditions, and location can also play a significant role in attracting and retaining scientists.
    It also does not account for scientists who have left the institution but have not left the field entirely, which may lead to an over-estimation of the institution's retention rate.

The formula for the retention rate is calculated as:

retention_rate = affiliated_scientists_prev_year / affiliated_scientists

where:

    affiliated_scientists_prev_year represents the number of scientists affiliated with the institution in the previous year
    affiliated_scientists represents the number of scientists affiliated with the institution in the current year.

In [ ]:
%%bigquery --project $project_id
CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2000_institutional_retention AS
WITH scientist_retention 
AS (
SELECT
research_orgs,
year,
researcher_ids,
LAG(research_orgs) OVER (PARTITION BY researcher_ids ORDER BY year) AS prev_research_orgs,
LAG(year) OVER (PARTITION BY researcher_ids ORDER BY year) AS prev_year,
FROM cshdimensionstest.test.researchers_after_1980_simplified
WHERE year >= 1980 AND year <= 2000
),
retention_rate AS (
SELECT
research_orgs,
year,
COUNT(DISTINCT researcher_ids) AS remaining_scientists
FROM scientist_retention
WHERE research_orgs = prev_research_orgs AND year = prev_year + 2 -- checks how many scientists remained affiliated in the previous 2 years
GROUP BY research_orgs, year
),
retention_rate_with_affiliation AS (
SELECT
retention_rate.research_orgs,
retention_rate.year,
retention_rate.remaining_scientists,
scientist_affiliation.affiliated_scientists,
CASE 
  WHEN scientist_affiliation.affiliated_scientists = 0 THEN 0
  ELSE retention_rate.remaining_scientists / scientist_affiliation.affiliated_scientists
END AS retention_rate
FROM retention_rate
JOIN (
SELECT
research_orgs,
year,
COUNT(DISTINCT researcher_ids) AS affiliated_scientists
FROM cshdimensionstest.test.researchers_after_1980_simplified
WHERE year >= 1980 AND year <= 2000
GROUP BY research_orgs, year
) AS scientist_affiliation
ON retention_rate.research_orgs = scientist_affiliation.research_orgs 
AND retention_rate.year = scientist_affiliation.year)
SELECT 
research_orgs,
year,
affiliated_scientists,
remaining_scientists,
retention_rate
FROM retention_rate_with_affiliation;

Query is running:   0%|          |

""


In [ ]:
%%bigquery --project $project_id
SELECT * 
FROM cshdimensionstest.test.flows_1980_2000_institutional_retention 
ORDER BY  research_orgs, year
LIMIT 30;

Query is running:   0%|          |

Downloading:   0%|          |

,research_orgs,year,affiliated_scientists,remaining_scientists,retention_rate
0,grid.1001.0,1981,252,42,0.166667
1,grid.1001.0,1982,339,60,0.176991
2,grid.1001.0,1983,392,71,0.181122
3,grid.1001.0,1984,512,81,0.158203
4,grid.1001.0,1985,548,99,0.180657
5,grid.1001.0,1986,597,103,0.172529
6,grid.1001.0,1987,760,106,0.139474
7,grid.1001.0,1988,841,155,0.184304
8,grid.1001.0,1989,880,149,0.169318
9,grid.1001.0,1990,970,137,0.141237


In [ ]:
%%bigquery --project $project_id
# merge the table with the main indicator table
CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2000_institutional_total_indicators AS
SELECT a.*, b.affiliated_scientists, b.remaining_scientists, retention_rate
FROM cshdimensionstest.test.flows_1980_2000_institutional_total_flow_indicators a
LEFT JOIN cshdimensionstest.test.flows_1980_2000_institutional_retention b 
ON a.research_orgs=b.research_orgs 
and a.year=b.year;

DROP TABLE IF EXISTS cshdimensionstest.test.flows_1980_2000_institutional_total_flow_indicators;
DROP TABLE IF EXISTS cshdimensionstest.test.flows_1980_2000_institutional_retention;

SELECT * 
FROM cshdimensionstest.test.flows_1980_2000_institutional_total_indicators
ORDER BY research_orgs, year asc
LIMIT 30;

Query is running:   0%|          |

Downloading:   0%|          |

,year,research_orgs,population_year,inflows,outflows,net_migration_rate,total_migrants,non_migrant_percentage,affiliated_scientists,affiliated_scientists_prev_year,retention_rate
0,1980,grid.1001.0,202,<NA>,<NA>,NaN,<NA>,NaN,<NA>,<NA>,NaN
1,1981,grid.1001.0,252,12,25,-5.15873,37,85.317460,252,202,0.801587
2,1982,grid.1001.0,339,46,40,1.76991,86,74.631268,339,252,0.743363
3,1983,grid.1001.0,392,75,76,-0.25510,151,61.479592,392,339,0.864796
4,1984,grid.1001.0,512,106,138,-6.25000,244,52.343750,512,392,0.765625
5,1985,grid.1001.0,548,146,160,-2.55474,306,44.160584,548,512,0.934307
6,1986,grid.1001.0,597,166,184,-3.01508,350,41.373534,597,548,0.917923
7,1987,grid.1001.0,760,253,212,5.39474,465,38.815789,760,597,0.785526
8,1988,grid.1001.0,841,278,314,-4.28062,592,29.607610,841,760,0.903686
9,1989,grid.1001.0,880,350,335,1.70455,685,22.159091,880,841,0.955682


* One thing that we notice is that the non-migrant percentage is negative.

* The reason why this happens is that we calculate the population number for each year and do not take into account the population that stayed from the previous year. We need to make some adjustments.

* To count only the distinct cumulative population for each year, you could add a column to your query that identifies whether a scientist is still affiliated with the same institution in the next year. Then, you can sum up the number of scientists who are still affiliated with the same institution and divide that by the total number of affiliated scientists to get the cumulative non-migrant percentage. 

* non-mobile = what is the percentage of non-mobile researchers who have never left up until that point in time

-- we need to change the `non_migrant_percentage `indicator and align it with the `retention_rate` indicators


### 3.3 Python

In [ ]:
from google.cloud import bigquery
client = bigquery.Client(project=project_id)

sql = """
  SELECT *
  FROM `cshdimensionstest.test.aggregated_moved_to_00_02` 
  order by geoid_o, date_o, date_d, catid_o
"""
movedto_edges = client.query(sql).to_dataframe()
movedto_edges.head(10)

# save the dataset
movedto_edges.to_csv('movedto_edges.csv')
files.download('movedto_edges.csv')

,geoid_o,geoid_d,catid_o,catid_d,date_o,date_d,date_range,weighted_flows,flows
0,grid.1001.0,grid.1003.2,2330,2366,2000,2001,2000-2002,1.000000,1
1,grid.1001.0,grid.32197.3e,2330,2933,2000,2001,2000-2002,1.000000,1
2,grid.1001.0,grid.508487.6,2330,2330,2000,2001,2000-2002,0.666667,2
3,grid.1001.0,grid.12136.37,2330,2409,2000,2001,2000-2002,0.666667,2
4,grid.1001.0,grid.264756.4,2330,2330,2000,2001,2000-2002,0.500000,1
5,grid.1001.0,grid.8127.c,2330,2447,2000,2001,2000-2002,0.833333,2
6,grid.1001.0,grid.117476.2,2330,2921,2000,2001,2000-2002,1.000000,2
7,grid.1001.0,grid.8484.0,2330,2746,2000,2001,2000-2002,0.333333,1
8,grid.1001.0,grid.5596.f,2330,2933,2000,2001,2000-2002,0.500000,1
9,grid.1001.0,grid.5333.6,2330,2921,2000,2001,2000-2002,0.333333,1


In [ ]:
from google.cloud import bigquery
client = bigquery.Client(project=project_id)

sql = """
  SELECT *
  FROM `cshdimensionstest.test.total_inflows_00_02` 
"""

inflows = client.query(sql).to_dataframe()

#inflows.to_csv('inflows.csv')
#!cp inflows.csv "gdrive/My Drive/CSH-DIMENSIONS Flows Test/BigQuery-results"

sql = """
  SELECT *
  FROM `cshdimensionstest.test.total_outflows_00_02` 
"""
outflows = client.query(sql).to_dataframe()

#from google.colab import files
#files.download('inflows.csv')
#outflows.to_csv('outflows.csv')
#!cp inflows.csv "gdrive/My Drive/CSH-DIMENSIONS Flows Test/BigQuery-results"
#from google.colab import files
#files.download('outflows.csv')

In [ ]:
#@title Hidden Cell
inflows.sort_values(["geoid_d", "catid_d", "date_d"]).head(10)

,geoid_d,catid_d,date_d,date_range,inflows,weightedInflows
4666,grid.1001.0,2330,2001,2000-2002,208,107.833333
4759,grid.1001.0,2330,2002,2000-2002,234,162.666667
7252,grid.1001.0,2344,2001,2000-2002,537,272.583333
9142,grid.1001.0,2344,2002,2000-2002,419,241.666667
12230,grid.1001.0,2353,2001,2000-2002,96,63.500000
11395,grid.1001.0,2353,2002,2000-2002,29,10.000000
16908,grid.1001.0,2358,2001,2000-2002,212,150.500000
16441,grid.1001.0,2358,2002,2000-2002,380,159.916667
18109,grid.1001.0,2366,2002,2000-2002,43,37.000000
20890,grid.1001.0,2377,2001,2000-2002,19262,2502.594061


In [ ]:
#@title Hidden Cell
outflows.sort_values(["geoid_o", "catid_o", "date_d"]).head(10)

,geoid_o,catid_o,date_d,date_range,t_outflows,t_weightedOutflows
4761,grid.1001.0,2330,2001,2000-2002,501,243.750000
3223,grid.1001.0,2330,2002,2000-2002,311,154.666667
9245,grid.1001.0,2344,2001,2000-2002,498,243.500000
9032,grid.1001.0,2344,2002,2000-2002,684,354.423077
11374,grid.1001.0,2353,2001,2000-2002,37,18.500000
12302,grid.1001.0,2353,2002,2000-2002,187,75.506410
15729,grid.1001.0,2358,2001,2000-2002,346,181.166667
14254,grid.1001.0,2358,2002,2000-2002,275,113.044872
17733,grid.1001.0,2366,2001,2000-2002,48,20.333333
17388,grid.1001.0,2366,2002,2000-2002,23,9.500000


In [ ]:
# merge the inflows and outflows dataframe
result = pd.merge(inflows
                  , outflows
                  , how="outer"
                  , left_on=["geoid_d", "catid_d", "date_d"]
                  , right_on=["geoid_o", "catid_o", "date_d"]
                  ).reset_index(drop = True)
def diff(a, b):
    return b - a

result["net_mobility"] = result['inflows'] - result['t_outflows']
result["weighted_net_mobility"] = result['weightedInflows'] - result['t_weightedOutflows']
#result.sort_values(["geoid_o", "catid_o", "date_d"]).head(10)
flow_ind = result.rename(columns = {'date_d': 'MoveYear'
                         , ' t_ouflows': 'outflows' 
                         , 't_weightedOutflows': 'weightedOutflows'
                         , 'date_range_x':'Range'
                         , 'net_mobility':'NetFlows'
                         , 'weighted_net_mobility': 'WeightedNetFlows'}) \
                         [[  'geoid_d', 'catid_d', 'inflows', 'weightedInflows'\
                           , 'geoid_o', 'catid_o', 't_outflows', 'weightedOutflows'\
                           , 'NetFlows', 'WeightedNetFlows', 'MoveYear', 'Range']]

# save the indicators to a csv file
#flow_ind.to_csv('Flows_indicators.csv')
#files.download('Flows_indicators.csv')
flow_ind.sort_values(["geoid_o", "catid_o", "MoveYear"]).head(10)

# store dataset directly into GBQ and DRIVE
# store in drive
flow_ind.to_csv('2023_01_08_flows_output.csv', encoding = 'utf-8-sig') 

# store in GBQ
# import pandas_gbq
# table_id = 'test.2023_01_08_flows_output'
# pandas_gbq.to_gbq(flow_ind, table_id, project_id=project_id)

flow_ind.head(1)

# PART III - Coverage

1. **Make a hello world program**
1. **Connect resources to each other:**
 e.g., can I print the GBQ data in a website (print=show any table) for instance?
1. **Other considerations**
* how to run queries fast enough (users should not have delays)
* how does the interface look like
* how to put all calculations in one query?
* how to connect the web interface to google bigquery?
* what if multiple users use it? performance?
